In [1]:
import os
import re
from time import sleep
import time
from bs4 import BeautifulSoup
import parsel
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import pymysql as db
from selenium.webdriver.common.action_chains import ActionChains


In [2]:
def start_search(driver, place_info):  #검색 시작 , searchbox에 계속 입력

    url = "https://www.google.com/maps/"
    driver.get(url)
    driver.set_window_position(800, 0)
    driver.set_window_size(700, 900)
    search = driver.find_element_by_css_selector("#searchboxinput")
    time.sleep(1)
    search.clear()
    search.send_keys(str(place_info['address']) + " "+str(place_info['name']))
    search.send_keys(Keys.ENTER)
    time.sleep(2)
    # driver.implicitly_wait(3)

    return get_store_review_data(driver, place_info)

In [3]:
def get_store_review_data(driver, place_info):
    try:
        driver.find_elements_by_css_selector('.hfpxzc')[0].click()    # 목록 생성시 처음 것 클릭
    except:
        pass
    time.sleep(2)
    review_count = 0
    #total_stars = driver.find_element_by_css_selector('.jANrlb').text.split('\n')[0] 
    try:
        review_count = int(re.sub(r'[^0-9]','',driver.find_element_by_css_selector('.jANrlb').text.split('\n')[1]))
        review_click = driver.find_element_by_css_selector('.DkEaL')           # 리뷰(개수) 버튼 클릭
        if '리뷰' not in review_click.text:                                   # 아니면 오류 발생
            raise
        else:
            review_click.click()
    except:
        try:
            click = driver.find_element_by_css_selector('.BgrMEd.cYlvTc')      # 리뷰 더보기 버튼 클릭
            if click.text in ['데이터 정보','사이트 방문','주변 호텔 더보기','다른 호텔 보기','질문 더보기']:
                print("review 없음")
                return None
            click.click()
        except:
            print("review 없음")
            return None
    #print(total_stars)
    print('리뷰개수 : ',review_count)
    if review_count > 1000:
        review_count = 1000             
    page_num = (review_count // 20) + 1 # 리뷰 개수 최대 500개, 뒤에는 이상한 리뷰들 혹은 별점만 부여한 리뷰들이 많음
    time.sleep(1)

    for i in range(page_num):           # 스크롤 내리기
        try :
            for j in range(11):
                driver.find_element_by_tag_name('.m6QErb.DxyBCb.kA9KIf.dS8AEf').send_keys(Keys.PAGE_DOWN) 
        except :
            print("scroll 끝")
            break
        time.sleep(1)
    
    # 스크롤 끝났으니 수집
    try:
        while True:
            driver.find_element_by_css_selector('.w8nwRe.kyuRq').click()     #더보기 버튼 계속 클릭
    except Exception as e:
        reviews = driver.find_elements_by_css_selector('.MyEned')
    
    #print('리뷰개수 : ' , len(reviews))
    result =[[place_info.get("id"), place_info.get("name"), review.text.replace("\n", " ")]     # 리뷰 저장해서 전달
            for review in reviews]
    print("수집끝")
    #print(result)
    return result

In [7]:
def main():

    driver = webdriver.Chrome()
    local = 'nexpot.c43ypw0liaa2.ap-northeast-2.rds.amazonaws.com'  # aws rds 접근
    con = db.connect(
        host=local,
        user='admin',
        db='Nexpot',
        password='qwertyuiop123',
        charset='utf8mb4'
    )
    cur = con.cursor()                             # score에 없는 placeID 입력
    cur.execute("select distinct placeID, pname, paddress from places where placeID not in (select score.placeID from places, score where places.placeID = score.placeID);")
    row_places = cur.fetchall()
    places = []
    for i in row_places:
        address = i[2][:2]
        places.append({
            'id' : i[0],
            'name' : i[1],
            'address' : address
        })
    print(len(places))
    f = open('google_review6.txt', "a", encoding='UTF-8')     # add 형식으로 txt 파일에 저장
    for i in range(len(places)):
        print(i)
        print(places[i]['id'])
        result = start_search(driver, places[i])              # 크롤링 시작
        if result is None:
            continue
        # 구, ID, stars, 리뷰
        # result = get_store_review_data(driver)
        data = pd.DataFrame(result)
        data.columns = ['placeID', 'placename', 'review']
        for k in range(len(data)):
            f.write(f"{data['placeID'][k]}\t{data['review'][k]}\n")
        #data.to_csv('google_reviews.csv', index=False, sep='|', mode="a"+, header=False, encoding="euc-kr")


In [8]:
main()

587
0
1251795966
리뷰개수 :  5
수집끝
1
989915294
리뷰개수 :  8
수집끝
2
1195698905
review 없음
3
15153456


WebDriverException: Message: chrome not reachable
  (Session info: chrome=105.0.5195.127)


In [20]:
# Test
local = 'nexpot.c43ypw0liaa2.ap-northeast-2.rds.amazonaws.com'  # aws rds 접근
con = db.connect(
    host=local,
    user='admin',
    db='Nexpot',
    password='qwertyuiop123',
    charset='utf8mb4'
)
cur = con.cursor()
cur.execute("select distinct placeID, pname, paddress from places where placeID not in (select score.placeID from places, score where places.placeID = score.placeID);")
row_places = cur.fetchall()
places = []
for i in row_places:
    if len(i[2]) < 3:
        address = i[2]
    else:
        address = i[2][:2]
    places.append({
        'id' : i[0],
        'name' : i[1],
        'address' : address
    })
print(len(places))

2230


In [140]:
#Test
driver = webdriver.Chrome()
url = "https://www.google.com/maps/"
driver.get(url)
driver.set_window_position(800, 0)
driver.set_window_size(700, 900)

search = driver.find_element_by_css_selector("#searchboxinput")
time.sleep(1.5)
search.clear()

search.send_keys(str(places[19]['address'])+" "+ str(places[19]['name']))
search.send_keys(Keys.ENTER)
time.sleep(1.5)
try:
    driver.find_elements_by_css_selector('.hfpxzc')[0].click()
except:
    pass
time.sleep(1.5)
#review_count = int(re.sub(r'[^0-9]','',driver.find_element_by_css_selector('.jANrlb').text.split('\n')[1]))
#total_stars = driver.find_element_by_css_selector('.jANrlb').text.split('\n')[0] 
#print(type(total_stars))

try:
    driver.find_element_by_css_selector('.DkEaL').click()
except:
    click = driver.find_element_by_css_selector('.BgrMEd.cYlvTc')
    if click.text == '데이터 정보':
        print("review 없음")
    print("review 없음")
    #break
#print(review_count)
if review_count > 1000:
    review_count = 1000
#for i in range(100):
#    try:
#        scroll = driver.find_element_by_css_selector(".e07Vkf.kA9KIf")
#        time.sleep(1)
#        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scroll)
#        location1 = scroll.location_once_scrolled_into_view
#        print(i, location1)
#    except Exception as e:
#        print(e)
#        break
page_num = (review_count // 20) + 1
time.sleep(1)

for i in range(page_num):  
    try :
        for j in range(13):
            driver.find_element_by_tag_name('.m6QErb.DxyBCb.kA9KIf.dS8AEf').send_keys(Keys.PAGE_DOWN)
    except :
        print("scroll 끝")
        break
    time.sleep(1)
    #time.sleep(1.5)
    print(i)
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#scrollable_div = driver.find_element_by_css_selector('div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc')
#driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)

try:
    while True:
        driver.find_element_by_css_selector('.w8nwRe.kyuRq').click()
except Exception as e:
    print(e)
    reviews = driver.find_elements_by_css_selector('.MyEned')
    stars = driver.find_elements_by_css_selector('.kvMYJc')
    print(len(reviews))
    print(type(stars))
    for r,s in zip(reviews,stars):
        print(r.text)
        print(s.text)
        print("#############")

0
1
2
3
4
5
6
7
8
9
Message: no such element: Unable to locate element: {"method":"css selector","selector":".w8nwRe.kyuRq"}
  (Session info: chrome=105.0.5195.127)

65
<class 'list'>
무창포해수욕장에서 갯벌이 열리는 시간은 하루2번 무조건 열렸다 닫히는데 다만 사리 기간중 갯벌이 많이 열리는날 (사리기간) 이 신비의바닷길이 열리는 날이랍니다
주로 봄에는 주간에 많이
열리고 가을에는 야간에 많이 열립니다

낙조1경-무창포타워

낙조2경-신비의바닷길 입구

낙조3경-다리위

낙조4경-흰등대

낙조5경-닭벼슬섬

또하나의 재미거리

바지락체험을 할수있는데요

개인 10,000원 단체 8,000원

호미 1000원 장화2000원의 대여로를 내고 빌릴수있어요

무창포에서는 봄에 쭈꾸미축제도 있고

가을엔 대하 전어축제

8~9월에 신비의바닷길 축제도 있다고하네요

시스케이프까페가 들어가면 생각보다 넓고 예쁜 뷰맛집이랍니다
편안하게 누워 닭벼슬섬과 다리야경을 보며 쥬스와 커피콩빵을 먹고 있으니 신선놀음이 따로 없어요ㅋ
일몰좋아요

#############
대천 해수욕장의 남쪽에 위치한 해수욕장.
관리 잘 되어 있습니다.

#############
우연히 이동중에 방문하게 된 모세의 기적으로 유명한 무창포해수욕장. 아쉽게도 2021년 모세의 기적은 12월8일로 종료되었습니다. 곳곳에 무료주차장이 마련되어 있지만 토요일이라 그런지 거의 만차였습니다. 축제기간중에는 주차난으로 고생할거 같습니다. 썰물때라 그런지 백사장이 넓은게 보기 좋습니다. 곳곳에 감시초소도 있고, 샤워장, 호미랑 장화 대여소도 있습니다. 주변에 다양한 음식점도 많이 있어서 좋았습니다. 민박집이나 호텔 등도 많이 있는데 비수기라 그런지 문닫은 곳이 많네요. 포토존도 곳곳에 있구요. 특이한건 500원 동전을 넣으면 2분간 가동하는 분수들이 몇군데 있습니다. 여름철에 